# Reduced Displacement calculation tutorial

# 1. Header

In [5]:
import os
import sys
import time
import obspy
from obspy.clients.filesystem.sds import Client as sdsclient
from obspy.core.inventory.inventory import read_inventory

PROJECT_DIR = os.path.join('..','..')
LIB_DIR = os.path.join(PROJECT_DIR,'src','lib')
sys.path.append(LIB_DIR)
from SAM import VSAM, VSEM, EMAG
import InventoryTools
import SDS

CONTINUOUS_DATA_DIR = os.path.join(PROJECT_DIR, 'data', 'continuous')
SDS_DIR = os.path.join(CONTINUOUS_DATA_DIR, 'SDS')
SDS_DIR2 = os.path.join(CONTINUOUS_DATA_DIR, 'SDS_Velocity')
SAM_DIR = os.path.join(CONTINUOUS_DATA_DIR, 'SAM')

#####################################################################################

startTime = obspy.core.UTCDateTime(2001,2,25,0,0,0)
endTime = obspy.core.UTCDateTime(2001,3,6,0,0,0)
secondsPerDay = 60 * 60 * 24
numDays = (endTime-startTime)/secondsPerDay


# 2. Write SDS_Velocity archive
- Read files from SDS_raw
- Correct to Displacement
- Write files to SDS_Velocity

In [6]:
def correct_seed_id(tr):
    net, sta, loc, chan = tr.id.split('.')
    Fs = tr.stats.sampling_rate
    code0 = chan[0]
    if Fs < 80.0 and Fs > 20.0:
        if chan[0] == 'E': 
            code0 = 'S'
        elif chan[0] == 'H': 
            code0 = 'B'
    elif Fs > 80.0 and Fs < 250.0:
        if chan[0] == 'B': 
            code0 = 'H'
        elif chan[0] == 'S': 
            code0 = 'E'
    chan = code0 + chan[1:] 

mySDSreadClient = sdsclient(SDS_DIR)
mySDSwriteClient = SDS.SDSobj(SDS_DIR2, sds_type='D', format='MSEED')

pre_filt = (0.01, 0.02, 18.0, 36.0)
invMVO = read_inventory('../../data/response/MV.xml', format='stationxml')   
inv_ids = InventoryTools.inventory2traceid(invMVO, force_location_code='')

smalltime = 0.01
daytime = startTime
taperSecs = 1800
while daytime < endTime:
    
    print(f'Loading Stream data for {daytime}')
    stMV = mySDSreadClient.get_waveforms("MV", "*", "*", "[SBEHCD]*", daytime-taperSecs, daytime+secondsPerDay+taperSecs)
    stMV.merge(method=0, fill_value=0)
    print(f'- got {len(stMV)} Trace ids') 
    for tr in stMV:

        correct_seed_id(tr)
        
        if tr.id in inv_ids:
            time1 = time.time()
            tr.remove_response(inventory=invMVO, pre_filt=pre_filt, output="VEL")
            this_st = obspy.core.Stream(traces=tr)
            this_st.trim(starttime=daytime, endtime=daytime+secondsPerDay-smalltime)
            mySDSwriteClient.stream = this_st
            mySDSwriteClient.write(overwrite=False, merge=False)
            time2 = time.time()
            print(this_st,' took ',time2-time1,' s')
        else:
            print(f"{tr.id} not found in MV.xml")

        print(' ')

    daytime += secondsPerDay
del mySDSreadClient
del mySDSwriteClient

Loading Stream data for 2001-02-25T00:00:00.000000Z
- got 18 Trace ids
write
1 Trace(s) in Stream:
MV.MBBY..BHE | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples
MV.MBBY..BHE | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples
../../data/continuous/SDS_Velocity/2001/MV/MBBY/BHE.D
sds.write:  ../../data/continuous/SDS_Velocity/2001/MV/MBBY/BHE.D/MV.MBBY..BHE.D.2001.056  does not already exist. Writing
1 Trace(s) in Stream:
MV.MBBY..BHE | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples  took  10.792600870132446  s
 


/home/thompsong/miniconda3/envs/iceweb/lib/python3.10/site-packages/obspy/io/mseed/core.py:770: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


write
1 Trace(s) in Stream:
MV.MBBY..BHN | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples
MV.MBBY..BHN | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples
../../data/continuous/SDS_Velocity/2001/MV/MBBY/BHN.D
sds.write:  ../../data/continuous/SDS_Velocity/2001/MV/MBBY/BHN.D/MV.MBBY..BHN.D.2001.056  does not already exist. Writing
1 Trace(s) in Stream:
MV.MBBY..BHN | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples  took  6.091328859329224  s
 
write
1 Trace(s) in Stream:
MV.MBBY..BHZ | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples
MV.MBBY..BHZ | 2001-02-25T00:00:00.000000Z - 2001-02-25T23:59:59.986667Z | 75.0 Hz, 6480000 samples
../../data/continuous/SDS_Velocity/2001/MV/MBBY/BHZ.D
sds.write:  ../../data/continuous/SDS_Velocity/2001/MV/MBBY/BHZ.D/MV.MBBY..BHZ.D.2001.056  does not already exist. Writing
1 Trace(s) in Stream:
MV.MBBY..BHZ | 2

# 3. Write VSAM archive
- read data from SDS_Velocity
- compute VSAM data
- write to VSAM files

In [7]:
mySDSreadClient = sdsclient(SDS_DIR2)
daytime = startTime
while daytime < endTime:
    time1 = time.time()
    print(f'Loading Stream data for {daytime}')
    stMV = mySDSreadClient.get_waveforms("MV", "*", "*", "[SBEHCD]*", daytime, daytime+secondsPerDay)
    print(f'- got {len(stMV)} Trace ids') 
    print(f'Computing VSAM metrics for {daytime}, and saving to pickle files')
    for tr in stMV:
        tr.stats['units'] = 'm'
    vsamMV24h = VSAM(stream=stMV, sampling_interval=60)
    vsamMV24h.write(SAM_DIR, ext='pickle')
    
    daytime += secondsPerDay

    time2=time.time()
    print(f"- day took {time2-time1} seconds")

del mySDSreadClient

Loading Stream data for 2001-02-25T00:00:00.000000Z
- got 12 Trace ids
Computing VSAM metrics for 2001-02-25T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 2.1902284622192383 seconds
Loading Stream data for 2001-02-26T00:00:00.000000Z
- got 12 Trace ids
Computing VSAM metrics for 2001-02-26T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 1.6959257125854492 seconds
Loading Stream data for 2001-02-27T00:00:00.000000Z
- got 12 Trace ids
Computing VSAM metrics for 2001-02-27T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 2.694896936416626 seconds
Loading Stream data for 2001-02-28T00:00:00.000000Z
- got 12 Trace ids
Computing VSAM metrics for 2001-02-28T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 1.8407959938049316 seconds
Loading Stream data for 2001-03-01T00:00:00.000000Z
- got 12 Trace ids
Computing VSAM metrics for 2001-03-01T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 2.247601

In [8]:
# 4. compute VSEM, with a method to compute EMAG
mySDSreadClient = sdsclient(SDS_DIR2)
daytime = startTime
while daytime < endTime:
    time1 = time.time()
    print(f'Loading Stream data for {daytime}')
    stMV = mySDSreadClient.get_waveforms("MV", "*", "*", "[SBEHCD]*", daytime, daytime+secondsPerDay)
    print(f'- got {len(stMV)} Trace ids') 
    print(f'Computing VSEM metrics for {daytime}, and saving to pickle files')
    for tr in stMV:
        tr.stats['units'] = 'm'
    vsemMV24h = VSEM(stream=stMV, sampling_interval=60)
    vsemMV24h.write(SAM_DIR, ext='pickle')
    emagMV24h = vsemMV24h.compute_emag()
    emagMV24h.write(SAM_DIR, ext='pickle')
    
    daytime += secondsPerDay

    time2=time.time()
    print(f"- day took {time2-time1} seconds")

del mySDSreadClient

Loading Stream data for 2001-02-25T00:00:00.000000Z
- got 12 Trace ids
Computing VSEM metrics for 2001-02-25T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 1.846832513809204 seconds
Loading Stream data for 2001-02-26T00:00:00.000000Z
- got 12 Trace ids
Computing VSEM metrics for 2001-02-26T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 2.1065425872802734 seconds
Loading Stream data for 2001-02-27T00:00:00.000000Z
- got 12 Trace ids
Computing VSEM metrics for 2001-02-27T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 2.276064395904541 seconds
Loading Stream data for 2001-02-28T00:00:00.000000Z
- got 12 Trace ids
Computing VSEM metrics for 2001-02-28T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 2.1380858421325684 seconds
Loading Stream data for 2001-03-01T00:00:00.000000Z
- got 12 Trace ids
Computing VSEM metrics for 2001-03-01T00:00:00.000000Z, and saving to pickle files
VSAM
write

- day took 2.4329469

In [9]:
# Load VSAM data
vsamMV = VSAM.read(startTime, endTime, SAM_DIR=SAM_DIR, sampling_interval=60, ext='pickle')

# Plot VSAM data
vsamMVhourly = vsamMV.downsample(new_sampling_interval=3600)
vsamMVhourlyZ = vsamMVhourly.select(component='Z')
for metric in ['median', 'mean', 'VLP', 'LP', 'VT', 'fratio']:
    vsamMVhourlyZ.plot(metric=metric, outfile=f'plots/VSAM_hourly.png')


../../data/continuous/SAM/VSAM_*_2001_60s.pickle
[]
[]
no valid dataframes found
creating blank SAM object
no valid dataframes found
creating blank SAM object
no valid dataframes found
creating blank SAM object


AttributeError: 'VSAM' object has no attribute 'iceweb_plot'

In [ ]:
# 5. compute EMAG
